In [59]:
import rasterio
import rasterio.plot
import pandas as pd
import matplotlib
import geopandas as gpd
import numpy as np
import rioxarray

####Objective####
# Join dataframes together
# Label data


# <<< LAND COVER MAP >>>
# Dimensions: 700000x1300000
LAND_COVER_MAP_PATH = 'Datasets/Digimap/a3ff9411-3a7a-47e1-9b3e-79f21648237d/data/gb2021lcm1km_percentage_target.tif'

# <<< FERTILISER >>>
# Dimensions: 651000x1216000
# The dataset consists of maps of the predicted average annual application rates (2010-2015) of three different inorganic 
# chemical fertilisers – nitrogen (N), phosphorus (P) and potassium (K) – in England across a six-year period, along with 
# their respective estimates of uncertainty, at a 1 km x 1 km resolution. 
FERTILISER_K_PATH = 'Datasets/Digimap/Land-Cover-plus-Fertilisers_4731497/fertiliser_k_prediction_uncertainty.tif'
FERTILISER_N_PATH = 'Datasets/Digimap/Land-Cover-plus-Fertilisers_4731497/fertiliser_n_prediction_uncertainty.tif'
FERTILISER_P_PATH = 'Datasets/Digimap/Land-Cover-plus-Fertilisers_4731497/fertiliser_p_prediction_uncertainty.tif'

# <<< PESTICIDE >>>
# Dimensions: 596000x1209000
PESTICIDE_PATH = 'Datasets\Digimap\Land-Cover-plus-Pesticides_4719434\Amidosulfuron.tif'


# <<< INTEGRATED HYDROLOGICAL DIGITAL TERRAIN MODEL >>>
# Dimensions: 700000x1300000
# These datasets all only have one band
ELEVATION_PATH = 'Datasets\Digimap\ihdtm-2016_4731496\HGHT_1km.tif'
CUMULATIVE_CATCHMENT_AREA_PATH = 'Datasets\Digimap\ihdtm-2016_4731496\CCAR_1km.tif'
SURFACE_TYPE_PATH = 'Datasets\Digimap\ihdtm-2016_4731496\SURF_1km.tif'
OUTFLOWING_DRAINAGE_DIRECTION_PATH = 'Datasets\Digimap\ihdtm-2016_4731496\OUTF_1km.tif'
INFLOWING_DRAINAGE_PATTERN_PATH = 'Datasets\Digimap\ihdtm-2016_4731496\INFL_1km.tif'

IHDTM = [ELEVATION_PATH, CUMULATIVE_CATCHMENT_AREA_PATH, SURFACE_TYPE_PATH, OUTFLOWING_DRAINAGE_DIRECTION_PATH, INFLOWING_DRAINAGE_PATTERN_PATH]


dataset = rioxarray.open_rasterio(LAND_COVER_MAP_PATH)
dataset.name = 'data'
df = dataset.to_dataframe().drop(columns='spatial_ref')
print(df.value_counts())
lcm_df = df.unstack(level='band')
lcm_df



# <<< Opening in rasterio >>>
# dataset = rasterio.open(FERTILISER_P_PATH)
# data = dataset.read()
# np.max(data)


data
0       18203710
1         123393
2          71220
3          51085
4          39528
          ...   
89          3191
93          3155
87          3143
82          3140
90          3129
Length: 101, dtype: int64


data                             ...                       \
band                 1  2  3  4  5  6  7  8  9  10  ... 12 13 14 15 16 17 18   
y         x                                         ...                        
1299500.0 500.0       0  0  0  0  0  0  0  0  0  0  ...  0  0  0  0  0  0  0   
          1500.0      0  0  0  0  0  0  0  0  0  0  ...  0  0  0  0  0  0  0   
          2500.0      0  0  0  0  0  0  0  0  0  0  ...  0  0  0  0  0  0  0   
          3500.0      0  0  0  0  0  0  0  0  0  0  ...  0  0  0  0  0  0  0   
          4500.0      0  0  0  0  0  0  0  0  0  0  ...  0  0  0  0  0  0  0   
...                 ... .. .. .. .. .. .. .. .. ..  ... .. .. .. .. .. .. ..   
500.0     695500.0    0  0  0  0  0  0  0  0  0  0  ...  0  0  0  0  0  0  0   
          696500.0    0  0  0  0  0  0  0  0  0  0  ...  0  0  0  0  0  0  0   
          697500.0    0  0  0  0  0  0  0  0  0  0  ...  0  0  0  0  0  0  0   
          698500.0    0  0  0  0  0  0  0  0  0  0  ...  0  0  0  0  0  0  0   
          699500.0    0  0  0  0  0  0  0  0  0  0  ...  0  0  0  0  0  0  0   

                             
band               19 20 21  
y         x                  
1299500.0 500.0     0  0  0  
          1500.0    0  0  0  
          2500.0    0  0  0  
          3500.0    0  0  0  
          4500.0    0  0  0  
...                .. .. ..  
500.0     695500.0  0  0  0  
          696500.0  0  0  0  
          697500.0  0  0  0  
          698500.0  0  0  0  
          699500.0  0  0  0  

[910000 rows x 21 columns]

In [57]:
LCM_CLASSES = [
    'Deciduous woodland', 
    'Coniferous woodland', 
    'Arable', 
    'Improve grassland', 
    'Neutral grassland', 
    'Calcareous grassland', 
    'Acid grassland',  
    'Fen', 
    'Heather', 
    'Heather grassland', 
    'Bog', 
    'Saltwater',
    'Freshwater',
    'Supralittoral rock',
    'Supralittoral sediment',
    'Littoral rock',
    'Littoral sediment',
    'Saltmarsh',
    'Urban',
    'Suburban'
    ]

test_df = lcm_df['data'].rename(columns={i+1: j for i, j in enumerate(LCM_CLASSES)})
test_df

band                Deciduous woodland  Coniferous woodland  Arable  \
y         x                                                           
1299500.0 500.0                      0                    0       0   
          1500.0                     0                    0       0   
          2500.0                     0                    0       0   
          3500.0                     0                    0       0   
          4500.0                     0                    0       0   
...                                ...                  ...     ...   
500.0     695500.0                   0                    0       0   
          696500.0                   0                    0       0   
          697500.0                   0                    0       0   
          698500.0                   0                    0       0   
          699500.0                   0                    0       0   

band                Improve grassland  Neutral grassland  \
y         x                                                
1299500.0 500.0                     0                  0   
          1500.0                    0                  0   
          2500.0                    0                  0   
          3500.0                    0                  0   
          4500.0                    0                  0   
...                               ...                ...   
500.0     695500.0                  0                  0   
          696500.0                  0                  0   
          697500.0                  0                  0   
          698500.0                  0                  0   
          699500.0                  0                  0   

band                Calcareous grassland  Acid grassland  Fen  Heather  \
y         x                                                              
1299500.0 500.0                        0               0    0        0   
          1500.0                       0               0    0        0   
          2500.0                       0               0    0        0   
          3500.0                       0               0    0        0   
          4500.0                       0               0    0        0   
...                                  ...             ...  ...      ...   
500.0     695500.0                     0               0    0        0   
          696500.0                     0               0    0        0   
          697500.0                     0               0    0        0   
          698500.0                     0               0    0        0   
          699500.0                     0               0    0        0   

band                Heather grassland  ...  Saltwater  Freshwater  \
y         x                            ...                          
1299500.0 500.0                     0  ...          0           0   
          1500.0                    0  ...          0           0   
          2500.0                    0  ...          0           0   
          3500.0                    0  ...          0           0   
          4500.0                    0  ...          0           0   
...                               ...  ...        ...         ...   
500.0     695500.0                  0  ...          0           0   
          696500.0                  0  ...          0           0   
          697500.0                  0  ...          0           0   
          698500.0                  0  ...          0           0   
          699500.0                  0  ...          0           0   

band                Supralittoral rock  Supralittoral sediment  Littoral rock  \
y         x                                                                     
1299500.0 500.0                      0                       0              0   
          1500.0                     0                       0              0   
          2500.0                     0                       0              0   
          3500.0             

In [ ]:

dataset = rioxarray.open_rasterio(ELEVATION_PATH)
dataset.name = 'data'
df = dataset.to_dataframe()

# Adding 25 to x and y coordinates to match other datasets
# Make this into a function for the other IHDTM datasets
df.index = df.index.set_levels(df.index.levels[1]+25, level=1)
df.index = df.index.set_levels(df.index.levels[2]+25, level=2)
df